In [1]:
import ast
import numpy as np
import sklearn.model_selection as sk
from whatwhy import QUESTION_WORDS
from whatwhy.text_processing.helper_methods import get_df_from_file
%run "/home/stevengt/Documents/code/whatwhy/notebooks/Tweet CSV Consolidation.ipynb"

In [1]:
import gensim.downloader as api
gensim_model = api.load("glove-wiki-gigaword-50")
# gensim_model = gensim.models.KeyedVectors.load_word2vec_format("/home/stevengt/Downloads/GoogleNews-vectors-negative300.bin", binary=True)
# gensim_model = gensim.downloader.load("glove-wiki-gigaword-50")

KeyboardInterrupt: 

In [1]:
import tensorflow as tf


# Configure keras to use the plaidml backend in order to use an AMD GPU.
import ngraph_bridge
from tensorflow.keras import backend as K
# ngraph_bridge.set_backend('PLAIDML')
config = tf.ConfigProto()

# backend_name = "PLAIDML" 
# device_id = "opencl_amd_hawaii.0" 
# opt_name = 'ngraph-optimizer' 
# from tensorflow.core.protobuf import rewriter_config_pb2
# rewriter_options = rewriter_config_pb2.RewriterConfig() 
# rewriter_options.meta_optimizer_iterations=(rewriter_config_pb2.RewriterConfig.ONE) 
# rewriter_options.min_graph_nodes=-1 
# ngraph_optimizer = rewriter_options.custom_optimizers.add() 
# ngraph_optimizer.name = opt_name 
# ngraph_optimizer.parameter_map["ngraph_backend"].s = backend_name.encode() 
# ngraph_optimizer.parameter_map["device_id"].s = device_id.encode() 
# config.MergeFrom(tf.ConfigProto(graph_options=tf.GraphOptions(rewrite_options=rewriter_options))) 


# config_ngraph_enabled = ngraph_bridge.update_config(config, backend_name="PLAIDML", device_id="opencl_amd_hawaii.0" ) 
with tf.Session(config=config_ngraph_enabled).as_default() as sess:
    K.set_session(sess)
    # from tensorflow.python.client import device_lib
    # print(device_lib.list_local_devices())
    print(sess.list_devices())

/home/stevengt/anaconda3/envs/whatwhy/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stevengt/anaconda3/envs/whatwhy/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stevengt/anaconda3/envs/whatwhy/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stevengt/anaconda3/envs/whatwhy

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 10533772564491166947), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5150765601722366644)]


/home/stevengt/anaconda3/envs/whatwhy/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stevengt/anaconda3/envs/whatwhy/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stevengt/anaconda3/envs/whatwhy/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stevengt/a

In [4]:
def get_text_as_list(text):
    if text is None or text is np.nan:
        return []
    else:
        return ast.literal_eval(text)

def get_max_num_tokens(all_tokens):
    return max( [ len(tokens_list) for tokens_list in all_tokens ] )

def get_max_token_length(all_tokens):
    longest_tokens = [0]
    for tokens_list in all_tokens:
        if len(tokens_list) != 0:
            longest_tokens.append( max([ len(token) for token in tokens_list ]) )
    return max(longest_tokens)

def get_num_words_in_vocab(gensim_model):
    return len(gensim_model.vocab.keys())

def unflatten_vector(flattened_vector, max_num_tokens, vector_size):
    return flattened_vector.reshape([max_num_tokens, vector_size])

def convert_embedded_phrase_to_english(gensim_model, embedded_phrase_vector, max_num_tokens):
    words = []
    vector_size = gensim_model.vector_size
    embedded_word_vectors = unflatten_vector(gensim_model, embedded_phrase_vector, max_num_tokens, vector_size)
    for i in range(max_num_tokens):
        embedded_word_vector = embedded_word_vectors[i,:]
        word = gensim_model.similar_by_vector(embedded_word_vector, topn=1)[0][0]
        words.append(word)
    return " ".join(words)

def convert_one_hot_gensim_model_indeces_to_english(gensim_model, one_hot_indeces, max_num_tokens):
    words = []
    vector_size = get_num_words_in_vocab(gensim_model)
    word_indeces = unflatten_vector(gensim_model, one_hot_indeces, max_num_tokens, vector_size)
    for i in range(max_num_tokens):
        word_index_one_hot = word_indeces[i,:]
        if np.count_nonzero(word_index_one_hot) > 0:
            word_index = np.argmax(word_index_one_hot)
            word = gensim_model.index2word[word_index]
            words.append(word)
    return " ".join(words)


def embed_list_of_tokens(gensim_model, tokens, max_num_tokens):
    vector_size = gensim_model.vector_size
    embedded_tokens = np.zeros([max_num_tokens, vector_size])
    cur_index = 0
    for token in tokens:
        try:
            embedded_tokens[cur_index, :] = gensim_model.get_vector(token)
            cur_index += 1
        except:
            pass
    return embedded_tokens

def embed_lists_of_tokens(gensim_model, tokens_lists):
    max_num_tokens = get_max_num_tokens(tokens_lists)
    num_token_lists = len(tokens_lists)
    num_tokens_per_sample = max_num_tokens
    token_vector_size = gensim_model.vector_size
    
    embedded_tokens = np.zeros([num_token_lists, num_tokens_per_sample, token_vector_size])

    for i, tokens_list in enumerate(tokens_lists):
        embedded_tokens[i,:,:] = embed_list_of_tokens(gensim_model, tokens_list, max_num_tokens)
    return embedded_tokens



def get_integer_indeces_from_tokens_lists(gensim_model, tokens_lists):
    max_num_tokens = get_max_num_tokens(tokens_lists)
    num_lists = len(tokens_lists)
    
    # Use a default value of -1 because 0 will be one-hot encoded, but not -1.
    indeces = -1 * np.ones([num_lists, max_num_tokens])
    for i, tokens_list in enumerate(tokens_lists):
        j = 0
        for token in tokens_list:
            try:
                indeces[i,j] = gensim_model.vocab[token].index
                j += 1
            except:
                continue
    return indeces

def get_flattened_one_hot_encodings_from_tokens_lists(gensim_model, tokens_lists):
    from tensorflow import one_hot
    max_num_tokens = get_max_num_tokens(tokens_lists)
    indeces = get_integer_indeces_from_tokens_lists(gensim_model, tokens_lists)
    depth = get_num_words_in_vocab(gensim_model)

    length_of_one_hot_of_multiple_tokens = depth * max_num_tokens
    encodings = -1 * np.ones([len(tokens_lists), length_of_one_hot_of_multiple_tokens])
    for i in range(len(tokens_lists)):
        encodings[i,:] = one_hot(indeces[i,:], depth).eval(session=sess).flatten()
    return encodings

def get_one_hot_encodings_from_tokens_lists(gensim_model, tokens_lists):
    from tensorflow import one_hot
    max_num_tokens = get_max_num_tokens(tokens_lists)
    indeces = get_integer_indeces_from_tokens_lists(gensim_model, tokens_lists)
    depth = get_num_words_in_vocab(gensim_model)

    length_of_one_hot_of_multiple_tokens = depth * max_num_tokens
    encodings = -1 * np.ones([len(tokens_lists), max_num_tokens, depth])
    for i in range(len(tokens_lists)):
        encodings[i,:,:] = one_hot(indeces[i,:], depth).eval(session=sess)
    return encodings

alphabet = "abcdefghijklmnopqrstuvwxyz"
char_to_int = dict((c,i+1) for i,c in enumerate(alphabet))
int_to_char = dict((i+1,c) for i,c in enumerate(alphabet))
int_to_char[0] = ""

def convert_to_onehot(data, max_num_characters):
    encoded_data = [char_to_int[char] for char in data]
    one_hot = np.zeros([max_num_characters, len(alphabet)])
    for i, value in enumerate(encoded_data):
        one_hot[i, value] = 1
    return np.asarray(one_hot)

# def get_flattened_one_hot_encodings_from_tokens_lists(tokens_lists, max_num_tokens, max_num_characters):
#     num_lists = len(tokens_lists)
#     depth = len(alphabet)
#     length_of_one_hot_of_multiple_tokens = depth * max_num_tokens * max_num_characters
#     encodings =  np.zeros([num_lists, length_of_one_hot_of_multiple_tokens])
#     for i in range(num_lists):
#         words = np.zeros([max_num_tokens, max_num_characters, depth])
#         for j in range(len(tokens_lists[i])):
#              words[j,:,:] = convert_to_onehot(tokens_lists[i][j], max_num_characters) 
#         encodings[i,:] = np.asarray(words).flatten()
#     return encodings

def get_text_from_flattened_one_hot(flattened_one_hot, max_num_tokens, max_num_characters):
    words = []
    unflattened_one_hot = flattened_one_hot.reshape([max_num_tokens, max_num_characters, len(alphabet)])
    num_words = unflattened_one_hot.shape[0]
    for i in range(num_words):
        word = ""
        for j in range(max_num_characters):
            index = np.argmax(unflattened_one_hot[i][j])
            word += int_to_char[index]
        words.append(word)
    return " ".join(words)

In [5]:
batch_dir = "/home/stevengt/Documents/code/whatwhy-data/News-Articles/all-the-news/tokens"
target_csv_name = "/home/stevengt/Documents/code/whatwhy-data/News-Articles/all-the-news/wh_phrases.csv"

df = get_df_from_file(target_csv_name)

for question_type in QUESTION_WORDS:
    token_col = question_type + " tokens"
    df[token_col] = df[token_col].apply(get_text_as_list)

In [6]:
all_what_tokens = df["what tokens"].tolist()[:5]
all_why_tokens = df["why tokens"].tolist()[:5]
all_tokens = all_what_tokens + all_why_tokens

In [7]:
max_num_what_tokens = get_max_num_tokens(all_what_tokens)
max_num_why_tokens = get_max_num_tokens(all_why_tokens)
max_num_what_characters = get_max_token_length(all_what_tokens)
max_num_why_characters = get_max_token_length(all_why_tokens)
print('Max number of what tokens: %d' % max_num_what_tokens)
print('Max number of why tokens: %d' % max_num_why_tokens)
print('Longest what word has %d letters' % max_num_what_characters)
print('Longest why word has %d letters' % max_num_why_characters)

Max number of what tokens: 4
Max number of why tokens: 8
Longest what word has 10 letters
Longest why word has 9 letters


In [8]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = sk.train_test_split(all_what_tokens, all_why_tokens, test_size=0.33, random_state = 42)

X_train = embed_lists_of_tokens(gensim_model, X_train_raw)
X_test = embed_lists_of_tokens(gensim_model, X_test_raw)

y_train = get_one_hot_encodings_from_tokens_lists(gensim_model, y_train_raw)
y_test = get_one_hot_encodings_from_tokens_lists(gensim_model, y_test_raw)

print(f"X_train.shape={X_train.shape}")
print(f"X_test.shape={X_test.shape}")
print(f"y_train.shape={y_train.shape}")
print(f"y_test.shape={y_test.shape}")


X_train.shape=(3, 4, 50)
X_test.shape=(2, 4, 50)
y_train.shape=(3, 4, 400000)
y_test.shape=(2, 8, 400000)


In [12]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed, Activation
from tensorflow.keras.optimizers import Adam

def define_model(gensim_model, max_num_what_tokens, max_num_why_tokens):
    vocabulary_size = get_num_words_in_vocab(gensim_model)
    hidden_size = gensim_model.vector_size
    input_shape = (max_num_what_tokens, hidden_size)
    output_shape = (max_num_why_tokens, vocabulary_size)
    use_dropout=False    
    
    model = Sequential()
    model.add(Input(shape=input_shape))
#     model.add(LSTM(hidden_size, return_sequences=True))
    model.add(LSTM(hidden_size, return_sequences=True))
    if use_dropout:
        model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(vocabulary_size)))
    model.add(Activation('softmax'))

    optimizer = Adam()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

    print(model.summary())
    return model

In [13]:
model=define_model(gensim_model, max_num_what_tokens, max_num_why_tokens)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4, 50)             20200     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 400000)         20400000  
_________________________________________________________________
activation_1 (Activation)    (None, 4, 400000)         0         
Total params: 20,420,200
Trainable params: 20,420,200
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(x=X_train, y=y_train, epochs=1, batch_size=16)
# save the model
# model.save('model.h5')

In [43]:
what_tokens = [
    ["hello", "world", "my", "name", "is", "steven"],
    ["this", "is", "not", "some", "drill"],
    ["this", "is", "only", "the", "test"]
]
why_tokens = [
    ["this", "is", "some", "sample", "sentence"],
    ["i", "learn", "with", "examples"],
    ["we", "must", "figure", "out", "some", "solution"]
]

max_num_what_tokens = get_max_num_tokens(what_tokens)
max_num_why_tokens = get_max_num_tokens(why_tokens)

max_num_what_characters = get_max_token_length(what_tokens)
max_num_why_characters = get_max_token_length(why_tokens)

# what_vecs = embed_lists_of_tokens(gensim_model, what_tokens)
why_one_hot = get_flattened_one_hot_encodings_from_tokens_lists(why_tokens, max_num_why_tokens, max_num_why_characters)

In [10]:
input_shape = (max_num_what_tokens, gensim_model.vector_size)
output_length = y_train.shape[1]
model = define_model(input_shape, output_length)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4, 50)]           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 4, 128)            58880     
_________________________________________________________________
dense (Dense)                (None, 4, 1872)           241488    
Total params: 300,368
Trainable params: 300,368
Non

In [ ]:
model.fit( X_train,
           y_train,
           batch_size=3,
           epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
